In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import time
import heapq
import numpy as np

# Specify the path to your Excel file
excel_file_path = 'C:\\Users\\Admin\\Documents\\Coding Portfolio\\Python\\Codes_and_Data_Thesis_2\\Codes_and_Data_Thesis_2\\Time_Matrix.xlsx'

# Read the Excel file into a pandas DataFrame without header
df = pd.read_excel(excel_file_path,sheet_name='Sheet1', header=None)

# Remove the header and frame
df = df.iloc[0:, :].reset_index(drop=True)

# Create a graph from the DataFrame
"""G = nx.Graph()
for i in range(len(df)):
    for j in range(len(df.iloc[i])):
        if df.iloc[i, j] != 0:
            G.add_edge(i, j, weight=df.iloc[i, j])"""
            
#Invisible duplicate graph for scanning other paths
G1 = nx.Graph()
for i in range(len(df)):
    for j in range(len(df.iloc[i])):
        if df.iloc[i, j] != 0:
            G1.add_edge(i, j, weight=df.iloc[i, j])

"""pos = nx.spring_layout(G1)
nx.draw(G1, pos, with_labels=True, font_weight='bold')
nx.draw_networkx_edge_labels(G1, pos, edge_labels={(u, v): d['weight'] for u, v, d in G1.edges(data=True)})"""

def dijkstra(graph, start):
    # Initialize distances with infinity for all nodes except the start node
    distances = {node: float('inf') for node in graph}
    distances[start] = 0

    # Initialize a dictionary to store the previous node in the shortest path
    previous_node = {node: None for node in graph}

    # Priority queue to store nodes with their current distance from start
    priority_queue = [(0, start)]
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        # Ignore outdated entries in the priority queue
        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            #print(current_distance)
            #print(weight['weight'])
            distance = current_distance + weight['weight']
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_node[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))
    # Construct the shortest paths
    shortest_paths = {node: [] for node in graph}
    for node in graph:
        current_node = node
        while previous_node[current_node] is not None:
            shortest_paths[node].insert(0, current_node)
            current_node = previous_node[current_node]
        if current_node == start:
            shortest_paths[node].insert(0, start)
    return distances, shortest_paths

# Find shortest paths from a source node (let's use node 0 as an example)
source_node = int(input("Enter your Starting Node: "))
# Get the shortest path from the source node to the last node in the critical path
destination_node = int(input("Enter your Destination Node: "))
start_time = time.time()
total_distance, shortest_path = dijkstra(G1, source_node)
print("Shortest Path: "+str(shortest_path[destination_node]))
print("Total Distance: "+str(total_distance[destination_node]))

alternative_shortest_path_length, alternative_shortest_path = dijkstra(G1, source_node)
other_distance = float(f"{alternative_shortest_path_length[destination_node]}")
#print(other_distance)
real_distance = float(total_distance[destination_node])
list_of_other_paths = []

#above code converted G to G1 except the seperate drawing of graph. Below are permanently G1

while True:
    #print(other_distance)
    if other_distance == real_distance:
        
        for i, current_item in enumerate(alternative_shortest_path[destination_node]):
            if i < len(alternative_shortest_path[destination_node]) - 1:  # Ensure we don't go out of bounds
                next_item = alternative_shortest_path[destination_node][i + 1]
                G1.remove_edge(current_item,next_item)
                #print("Current Item:", current_item)
                #print("Next Item:", next_item)
        new_shortest_path_length, new_shortest_path = dijkstra(G1, source_node)
        #print(new_shortest_path_length)
        #print(new_shortest_path)
        # Value to check
        """value_to_check = destination_node
        
        # Check if the value exists in any of the arrays in the dictionary values
        value_exists = any(value_to_check == value for array in new_shortest_path_length.values() for value in array)
        
        if not value_exists:
            break # no more other path towards the destination nodes"""
        other_distance = new_shortest_path_length[destination_node]
        if other_distance == real_distance:
            list_of_other_paths.append(new_shortest_path[destination_node])
        #print(str(other_distance)+"Bago distance pre")
        alternative_shortest_path_length, alternative_shortest_path = dijkstra(G1, source_node)
    else:
        break
#new_shortest_path_length, new_shortest_path = dijkstra(G1, source_node)
#print(new_shortest_path)
print("Other Paths: "+str(list_of_other_paths))
end_time = time.time()
final_time = end_time - start_time

print("Elapsed time: ", final_time)

Enter your Starting Node:  7
Enter your Destination Node:  20


Shortest Path: [7, 6, 9, 5, 16, 18, 20]
Total Distance: 290.776
Other Paths: []
Elapsed time:  0.010773897171020508
